<a href="https://colab.research.google.com/github/AlmTechSoftware/piper/blob/main/training_data/research/feynman_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl http://0x0.st/HLoi.zip > dataset.zip
!unzip dataset.zip -d dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4149k  100 4149k    0     0  2720k      0  0:00:01  0:00:01 --:--:-- 2719k
Archive:  dataset.zip
 extracting: dataset/README.dataset.txt  
 extracting: dataset/README.roboflow.txt  
   creating: dataset/test/
 extracting: dataset/test/Adam-still_jpeg.rf.53ade3c8a458fcd304c5a84d110e157c.jpg  
 extracting: dataset/test/Board_jpg.rf.9a7df51cf2743f861090462e582fd7ef.jpg  
 extracting: dataset/test/MPC_2_jpg.rf.0cd26e8fc0385c3e9c9bc14d7dcd8c94.jpg  
 extracting: dataset/test/_annotations.coco.json  
 extracting: dataset/test/falling_mp4-4_jpg.rf.7a0e2249c9ce2223be33120328072dab.jpg  
   creating: dataset/train/
 extracting: dataset/train/15525772766_3abe50a63a_jpg.rf.27a169a78b8d38bc783fa79fb69d112b.jpg  
 extracting: dataset/train/15525772766_3abe50a63a_jpg.rf.70a11a4dac447e271f7d7af832c0d65f.jpg  
 extracting: dataset/train/15525

In [2]:
!pip install  opencv-python \
              scikit-image \
              python-dotenv \
              pycocotools \
              dataclasses-json \
              supervision \
              colored \
              wandb \
              torchvision -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


## WandB login

In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Dataset Handler Definition

In [4]:
import os
from typing import Tuple
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
import pycocotools.coco as coco

from PIL import Image
from matplotlib import cm

class COCODataset(Dataset):
    def __init__(
        self,
        dataset_dir: str,
    ):
        self.dataset_dir = dataset_dir
        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(
                    # NOTE: ImageNet
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225],
                ),
            ]
        )

        self.coco_data = self.load_coco_data()

    def load_coco_data(self):
        coco_file_path = os.path.join(self.dataset_dir, "_annotations.coco.json")
        return coco.COCO(coco_file_path)

    def __len__(self):
        return len(self.coco_data.imgs)

    def __getitem__(self, idx):
        image_info = list(self.coco_data.imgs.values())[idx]
        image_path = os.path.join(self.dataset_dir, image_info["file_name"])

        # Load and preprocess the image
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)

        # Load and preprocess the segmentation mask
        ann_ids = self.coco_data.getAnnIds(imgIds=image_info["id"], iscrowd=None)
        mask = np.zeros((image_info["height"], image_info["width"]))
        for ann_id in ann_ids:
            mask += self.coco_data.annToMask(self.coco_data.anns[ann_id])
        mask = torch.tensor(mask).unsqueeze(0).float() / 255.0
        mask = mask.repeat(3, 1, 1)

        return image, mask

In [5]:
import torchvision.transforms as T
_tensor_pil_transform = T.ToPILImage()

def tensor_to_pil(tensor: torch.Tensor):
    return _tensor_pil_transform(tensor)

### Example

In [7]:
dataset = COCODataset("dataset/train")

sak = dataset[0]
display(sak)
img, mask = sak
img = tensor_to_pil(img)
mask = tensor_to_pil(mask)
display()

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


(tensor([[[ 0.8447,  0.9817,  1.0502,  ...,  2.0092,  2.0434,  2.0777],
          [ 0.6906,  0.7762,  0.8104,  ...,  2.0263,  2.0777,  2.1119],
          [ 0.5193,  0.5536,  0.5364,  ...,  2.0605,  2.0948,  2.1290],
          ...,
          [-0.5596, -0.4054, -0.3198,  ...,  1.2214,  1.4440,  1.9920],
          [-0.5082, -0.4226, -0.3541,  ...,  0.9303,  1.2214,  1.8550],
          [-0.5596, -0.5424, -0.4568,  ...,  0.6049,  0.9474,  1.6495]],
 
         [[ 1.0805,  1.2206,  1.2906,  ...,  2.2010,  2.2360,  2.2710],
          [ 0.9230,  1.0105,  1.0455,  ...,  2.2185,  2.2710,  2.3060],
          [ 0.7479,  0.7829,  0.7654,  ...,  2.2535,  2.2885,  2.3235],
          ...,
          [-0.5651, -0.4076, -0.3550,  ...,  1.1155,  1.3431,  1.9034],
          [-0.4601, -0.3725, -0.3550,  ...,  0.7654,  1.0630,  1.7108],
          [-0.5126, -0.4951, -0.4601,  ...,  0.4328,  0.7829,  1.5007]],
 
         [[ 1.4025,  1.5420,  1.6117,  ...,  2.5006,  2.5354,  2.5703],
          [ 1.2457,  1.3328,

TypeError: ignored